A script to transform a json file which is composed by a list of dictionaries, into a dictionary of dictionaries. The first "value" (with key = "collecting institution") of the previous dictionaries is set as the key for each new dictionary, and the values are the rest of the key/value pairs. Also included new keys and values based on previous data (which is very dependant to this precise requeriment)

In [ ]:
import json
import os

In [ ]:
with open("/home/pmata/git_repositories/laboratory_address.json") as file:
    orilist = json.load(file)
    final_dict={}
    for x in orilist:
        first_val = list(x.values())[0]
        x["submitting_institution"] = x["collecting_institution"]
        x["submitting_institution_address"] = x["collecting_institution_address"]
        x["submitting_institution_email"] = x["collecting_institution_email"]
        x.pop("collecting_institution")
        final_dict[first_val] = x
print(final_dict)

In [8]:
with open("/home/pmata/git_repositories/relecov-tools/relecov_tools/conf/laboratory_address2.json", "w", encoding="utf-8") as f:
    json.dump(final_dict, f, ensure_ascii=False, indent=4)

Checking if any of the records in the result file after using relecov-tools read-lab-metadata.py has no missing fields

In [5]:
with open("/tmp/relecov/processed_metadata_lab_COD-2129-AND-FIBAO_20230221.json") as file:
    newlist = json.load(file)
    original = len(newlist[0].keys())
    for x in newlist:
        if len(x.keys()) != original:
            print("ERROR")

ERROR


In [49]:
import relecov_tools.utils
from relecov_tools.config_json import ConfigJson

In [36]:
config_json = ConfigJson()
sftp_port = config_json.get_topic_data("sftp_handle", "sftp_port")
print(sftp_port)

22


In [40]:
metadata_tmp_folder = config_json.get_topic_data(
            "sftp_handle", "allowed_R1_delimiters"
                    )
print(metadata_tmp_folder)

['_R1_', '_R1.', '.R1.', 'Forward', 'forward', '_1_', '_1.', '.1.']


---

In [164]:
import json
import os
import relecov_tools.utils

In [128]:
def schema_dicter(schema):
    schema_json = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/{schema}")
    schema_dict = {}
    for key,value in schema_json["properties"].items():
        schema_dict[key] = {x:y for x,y in value.items() if x == "ontology" or x == "label"}
    label_dict = {}
    for x in schema_dict.values():
        for key,value in x.items():
            if key == "label":
                label_dict[value] = x["ontology"]
    return label_dict 

In [143]:
phagedict = schema_dicter("phage_schema.json")
enadict = schema_dicter("ena_schema.json")
reledict = schema_dicter("relecov_schema.json")
gisaidict = schema_dicter("gisaid_schema.json")
dictlist = {"phage": phagedict, "ena": enadict ,"relecov": reledict, "gisaid":gisaidict}

In [241]:
def schema_differ(schema1, schema2):
    same_ontologies = {k: schema1[k] for k in schema1 if k in schema2 and schema1[k] == schema2[k]}
    diff_ontologies = {k: schema1[k] for k in schema1 if k in schema2 and schema1[k] != schema2[k]}
    
    return same_ontologies, diff_ontologies

In [242]:
print(len(schema_differ(enadict, reledict)[0]), schema_differ(enadict, reledict)[1])
print(len(schema_differ(enadict, phagedict)[0]), schema_differ(enadict, phagedict)[1])
print(len(schema_differ(enadict, gisaidict)[0]), schema_differ(enadict, gisaidict)[1])
print(len(schema_differ(reledict, gisaidict)[0]), schema_differ(reledict, gisaidict)[1])
print(len(schema_differ(reledict, phagedict)[0]), schema_differ(reledict, phagedict)[1])
print(len(schema_differ(gisaidict, phagedict)[0]), schema_differ(gisaidict, phagedict)[1])

29 {}
14 {'Sample ID given by originating laboratory': 'GENEPIO:0001123', 'Sample Received Date': 'NCIT:C93644', 'Organism': 'NCIT:C43459'}
9 {'Additional Host Information': 'NCIT:C83280'}
14 {}
67 {'Sample ID given by originating laboratory': 'GENEPIO:0001123', 'Sample Received Date': 'NCIT:C93644', 'Organism': 'NCIT:C43459', 'Consensus sequence name': 'GENEPIO:0001461'}
14 {'Sample ID given by originating laboratory': 'GENEPIO:0001123'}


In [156]:
print(len(schema_differ(enadict, reledict)[0]), schema_differ(enadict, reledict)[1])
print(len(schema_differ(enadict, phagedict)[0]), schema_differ(enadict, phagedict)[1])
print(len(schema_differ(enadict, gisaidict)[0]), schema_differ(enadict, gisaidict)[1])
print(len(schema_differ(reledict, gisaidict)[0]), schema_differ(reledict, gisaidict)[1])
print(len(schema_differ(reledict, phagedict)[0]), schema_differ(reledict, phagedict)[1])
print(len(schema_differ(gisaidict, phagedict)[0]), schema_differ(gisaidict, phagedict)[1])

#Debugging in-place with import pdb; pdb.set_trace()


29 {}
14 {'Sample ID given by originating laboratory': 'GENEPIO:0001123', 'Sample Received Date': 'NCIT:C93644', 'Organism': 'NCIT:C43459'}
9 {'Additional Host Information': 'NCIT:C83280'}
14 {}
67 {'Sample ID given by originating laboratory': 'GENEPIO:0001123', 'Sample Received Date': 'NCIT:C93644', 'Organism': 'NCIT:C43459', 'Consensus sequence name': 'GENEPIO:0001461'}
14 {'Sample ID given by originating laboratory': 'GENEPIO:0001123'}


In [233]:
def schema_enumer(schema1, schema2):
    schema_json1 = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/{schema1}")
    schema_json2 = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/{schema2}")
    schema_dict1 = {}
    for key,value in schema_json1["properties"].items():
        if "Enums" in value.keys():
            schema_dict1[key] = [y for x,y in value.items() if x == "Enums"]
    schema_dict2 = {}
    for key,value in schema_json2["properties"].items():
        if "Enums" in value.keys():
            schema_dict2[key] = [y for x,y in value.items() if x == "Enums"]

    enums1 = [x[:-1].split(" [") for k,y in schema_dict1.items() for x in y[0]]
    enums2 = [x[:-1].split(" [") for k,y in schema_dict2.items() for x in y[0]]

    same_enums = {x[0]: [x[1],y[1]] for x in enums1 for y in enums2 if x[0] == y[0] and x[1] == y[1]}
    diff_enums = {x[0]: [x[1],y[1]] for x in enums1 for y in enums2 if x[0] == y[0] and x[1] != y[1]}

    return same_enums, diff_enums


In [234]:
phagedict2 = "phage_schema.json"
enadict2 = "ena_schema.json"
reledict2 = "relecov_schema.json"
gisaidict2 = "gisaid_schema.json"

In [235]:
print(len(schema_enumer(enadict2, reledict2)[0]), schema_enumer(enadict2, reledict2)[1])
print(len(schema_enumer(enadict2, phagedict2)[0]), schema_enumer(enadict2, phagedict2)[1])
print(len(schema_enumer(enadict2, gisaidict2)[0]), schema_enumer(enadict2, gisaidict2)[1])
print(len(schema_enumer(reledict2, gisaidict2)[0]), schema_enumer(reledict2, gisaidict2)[1])
print(len(schema_enumer(reledict2, phagedict2)[0]), schema_enumer(reledict2, phagedict2)[1])
print(len(schema_enumer(gisaidict2, phagedict2)[0]), schema_enumer(gisaidict2, phagedict2)[1])


399 {}
401 {}
20 {}
19 {}
520 {'Research': ['GENEPIO:0100022', 'GENEPIO:0100003'], 'Skin': ['UBERON:0001052', 'UBERON:0001003'], 'Middle Nasal Turbinate': ['UBERON:0001762', 'UBERON:0005921'], 'Air filter': ['GENEPIO:0100031', 'ENVO:00003968'], 'Blood Collection Tube': ['GENEPIO:0100031', 'OBI:0002859'], 'Bronchoscope': ['NCIT:C17611', 'OBI:0002826'], 'Collection Container': ['NCIT:C43446', 'OBI:0002088'], 'Filter': ['NCIT:C45801', 'GENEPIO:0100103'], 'Needle': ['NCIT:C69013', 'OBI:0000436'], 'Serum Collection Tube': ['NCIT:C113675', 'OBI:0002860'], 'Sputum Collection Tube': ['GENEPIO:0002115', 'OBI:0002861'], 'Suction Catheter': ['58253008', 'OBI:0002831'], 'Needle Biopsy': ['OBI:0002654', 'OBI:0002651'], 'Filtration': ['NCIT:C16583', 'OBI:0302885'], 'Lavage': ['NCIT:C38068', 'OBI:0600044'], 'Necropsy': ['NCIT:C166270', 'MMO:0000344'], 'Swabbing': ['NCIT:C17627', 'GENEPIO:0002117'], 'Inferior Nasal Turbinate': ['NCIT:C32794', 'UBERON:0005922'], 'Biopsy': ['NCIT:C15189', 'OBI:0002650']

In [ ]:
jsonprueba = relecov_tools.utils.read_json_file(f"/home/pmata/git_repositories/relecov-tools/relecov_tools/schema/ena_schema.json")
schemapru = {}
for key,value in jsonprueba["properties"].items():
    if "Enums" in list(value.keys()):
        schemapru[key] = [y for x,y in value.items() if x == "Enums"]

[x[:-1].split(" [") for k,y in schemapru.items() for x in y[0]]


In [ ]:
for x in schemapru.values():
    print(x)
    print(x[0][0].split("["))